# Import Libraries

In [247]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import pickle

In [152]:
import nltk

import re 

from nltk.stem import PorterStemmer # for stemming

from nltk.stem import WordNetLemmatizer # for lemmatization

from nltk.corpus import stopwords
nltk.download('punkt')

nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\92303\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\92303\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\92303\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# LOAD THE DATASET

In [153]:
data = pd.read_csv("gender-classifier.csv")

In [154]:
data

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,���It felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20045,815757572,True,golden,259,NaN,female,1.0000,yes,1.0,8/5/15 21:16,...,https://pbs.twimg.com/profile_images/656793310...,0,C0DEED,"@lookupondeath ...Fine, and I'll drink tea too...",NaN,783,10/26/15 13:20,6.587400e+17,Verona ���,NaN
20046,815757681,True,golden,248,NaN,male,1.0000,yes,1.0,8/15/12 21:17,...,https://pbs.twimg.com/profile_images/639815429...,0,0,Greg Hardy you a good player and all but don't...,NaN,13523,10/26/15 12:40,6.587300e+17,"Kansas City, MO",NaN
20047,815757830,True,golden,264,NaN,male,1.0000,yes,1.0,9/3/12 1:17,...,https://pbs.twimg.com/profile_images/655473271...,0,C0DEED,You can miss people and still never want to se...,NaN,26419,10/26/15 13:20,6.587400e+17,Lagos Nigeria,NaN
20048,815757921,True,golden,250,NaN,female,0.8489,yes,1.0,11/6/12 23:46,...,https://pbs.twimg.com/profile_images/657716093...,0,0,@bitemyapp i had noticed your tendency to pee ...,NaN,56073,10/26/15 12:40,6.587300e+17,Texas Hill Country,NaN


# PRE-PROCESSING THE DATASET

In [155]:
data.shape

(20050, 26)

In [156]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

In [157]:
data.isnull().sum()

_unit_id                     0
_golden                      0
_unit_state                  0
_trusted_judgments           0
_last_judgment_at           50
gender                      97
gender:confidence           26
profile_yn                   0
profile_yn:confidence        0
created                      0
description               3744
fav_number                   0
gender_gold              20000
link_color                   0
name                         0
profile_yn_gold          20000
profileimage                 0
retweet_count                0
sidebar_color                0
text                         0
tweet_coord              19891
tweet_count                  0
tweet_created                0
tweet_id                     0
tweet_location            7484
user_timezone             7798
dtype: int64

# Choosing only the required columns

In [158]:
df = data[['gender', 'description', 'text']]

In [159]:
df

,gender,description,text
0,male,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,I'm the author of novels filled with family dr...,���It felt like they were my friends and I was...
2,male,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...
...,...,...,...
20045,female,(rp),"@lookupondeath ...Fine, and I'll drink tea too..."
20046,male,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...
20047,male,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...
20048,female,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...


# Checking the Null Values

In [160]:
df.isnull().sum()

gender           97
description    3744
text              0
dtype: int64

# Removing the NULL VALUES

In [161]:
df.dropna(axis = 0, inplace = True)

In [162]:
df.isnull().sum()

gender         0
description    0
text           0
dtype: int64

In [163]:
df.shape

(16224, 3)

In [164]:
df['gender'].value_counts()

female     5725
male       5469
brand      4328
unknown     702
Name: gender, dtype: int64

In [165]:
df.dtypes

gender         object
description    object
text           object
dtype: object

In [166]:
df[df['gender'] == 'brand']

,gender,description,text
6,brand,"A global marketplace for images, videos and mu...",@BpackEngineer Thank you for your patience whi...
10,brand,Clean food that tastes great while providing e...,how are you taking care of yourself? https://t...
11,brand,highly extraordinary auctions,MTG Deals 1x Rank-Up-Magic - The Seventh One -...
13,brand,Come join the fastest blog network online toda...,https://t.co/z4sbWUugd8 What the Nation Will B...
26,brand,If you have any questions about Islam and woul...,How beautiful is the religion which teaches yo...
...,...,...,...
20030,brand,Crowdsourcing #Innovation! The FINND connects ...,The White House Is Ready For Tech To Step Up I...
20034,brand,Where Raleigh's dynamic leaders come to connec...,View the Community Halloween Spooktacular @Cit...
20038,brand,The FUN site for Animal Lovers - Get the lates...,Jon Stewart's Animal Sanctuary And Educational...
20041,brand,"When families go through divorce, it's helpful...",For Stan and Lisa... has #Coffee Cup at https:...


# Choosing only Male and Female from the Gender Column

In [167]:
df['gender'] = df[(df['gender'] == 'female') | (df['gender'] == 'male')]

In [168]:
df['gender'].value_counts()


female    5725
male      5469
Name: gender, dtype: int64

# Label Encoding, Representing Male with 1  and Female with 0

In [169]:
for gen in df['gender']:

  if gen=='male':

     df['gender'].replace({'male':1},inplace=True)

  elif gen=='female':

     df['gender'].replace({'female':0},inplace=True)

df['gender'].value_counts()


0.0    5725
1.0    5469
Name: gender, dtype: int64

# Cleaning the Description Column removing punctuation marks 

In [170]:
def clean(review):

    descrip = re.sub('[^a-zA-Z]', ' ', review)

    review = review.lower()

    return review

df['descrip_Cleaned'] = pd.DataFrame(df['description'].apply(lambda x: clean(x)))

df.head()


,gender,description,text,descrip_Cleaned
0,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1.0,I'm the author of novels filled with family dr...,���It felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1.0,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0.0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...


In [171]:
df['descrip_Cleaned'].replace('[@+]', "", regex=True,inplace=True)

df['descrip_Cleaned'].replace('[()]', "", regex=True,inplace=True)


df['descrip_Cleaned'].replace('[#+]', "", regex=True, inplace = True)


#url_regex = "(https?://)(s)*(www.)?(s)*((w|s)+.)*([w-s]+/)*([w-]+)((?)?[ws]*=s*[w%&]*)*"
#df['descrip_Cleaned'] = df['descrip_Cleaned'].replace(url_regex, "", regex=True)

In [172]:
#url_regex = "(https?://)(s)*(www.)?(s)*((w|s)+.)*([w-s]+/)*([w-]+)((?)?[ws]*=s*[w%&]*)*"
#df['descrip_Cleaned'] = df['descrip_Cleaned'].replace(url_regex, "", regex=True)

In [173]:
df['descrip_Cleaned'] = df['descrip_Cleaned'].replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='',regex=True)

In [174]:
df['descrip_Cleaned'] = df['descrip_Cleaned'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])


In [175]:
df

,gender,description,text,descrip_Cleaned
0,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1.0,I'm the author of novels filled with family dr...,���It felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1.0,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0.0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...
...,...,...,...,...
20045,0.0,(rp),"@lookupondeath ...Fine, and I'll drink tea too...",rp
20046,1.0,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...,"whatever you like, it's not a problem at all. ..."
20047,1.0,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...,teambarcelona ..you look lost so you should fo...
20048,0.0,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...,anti-statist; i homeschool my kids. aspiring t...


# Text Cleaning of the TEXT COLUMN

In [176]:
def clean1(review1):

    descrip = re.sub('[^a-zA-Z]', ' ', review1)

    review1 = review1.lower()

    return review1


In [177]:
df['text_Cleaned'] = pd.DataFrame(df['text'].apply(lambda y: clean1(y)))

df.head()

,gender,description,text,descrip_Cleaned,text_Cleaned
0,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.,robbie e responds to critics after win against...
1,1.0,I'm the author of novels filled with family dr...,���It felt like they were my friends and I was...,i'm the author of novels filled with family dr...,���it felt like they were my friends and i was...
2,1.0,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe...",hi @jordanspieth - looking at the url - do you...
4,0.0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...,watching neighbours on sky+ catching up with t...


In [178]:
df['text_Cleaned'].replace('[@+]', "", regex=True,inplace=True)

df['text_Cleaned'].replace('[()]', "", regex=True,inplace=True)


df['text_Cleaned']= df['text_Cleaned'].replace('[#+�۝]', "", regex=True)

In [179]:
df['text_Cleaned'] = df['text_Cleaned'].replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='',regex=True)

In [180]:
df['text_Cleaned'] = df['text_Cleaned'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])


In [181]:
url_regex = "(https?://)" 
df['text_Cleaned'] = df['text_Cleaned'].replace(url_regex, "", regex=True)

In [182]:
df

,gender,description,text,descrip_Cleaned,text_Cleaned
0,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.,robbie e responds to critics after win against...
1,1.0,I'm the author of novels filled with family dr...,���It felt like they were my friends and I was...,i'm the author of novels filled with family dr...,it felt like they were my friends and i was li...
2,1.0,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe...",hi jordanspieth - looking at the url - do you ...
4,0.0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...,watching neighbours on sky catching up with th...
...,...,...,...,...,...
20045,0.0,(rp),"@lookupondeath ...Fine, and I'll drink tea too...",rp,"lookupondeath ...fine, and i'll drink tea too...."
20046,1.0,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...,"whatever you like, it's not a problem at all. ...",greg hardy you a good player and all but don't...
20047,1.0,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...,teambarcelona ..you look lost so you should fo...,you can miss people and still never want to se...
20048,0.0,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...,anti-statist; i homeschool my kids. aspiring t...,bitemyapp i had noticed your tendency to pee o...


# REMOVING STOP WORDS

In [183]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [184]:
df['text_Cleaned'] = df['text_Cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [185]:
df

,gender,description,text,descrip_Cleaned,text_Cleaned
0,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.,robbie e responds critics win eddie edwards wo...
1,1.0,I'm the author of novels filled with family dr...,���It felt like they were my friends and I was...,i'm the author of novels filled with family dr...,felt like friends living story
2,1.0,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all,absolutely adore louis starts songs hits hard ...
3,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe...",hi jordanspieth - looking url - use ifttt?! ty...
4,0.0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...,watching neighbours sky catching neighbs!! xxx...
...,...,...,...,...,...
20045,0.0,(rp),"@lookupondeath ...Fine, and I'll drink tea too...",rp,"lookupondeath ...fine, i'll drink tea too. lov..."
20046,1.0,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...,"whatever you like, it's not a problem at all. ...",greg hardy good player get face dez bryant. le...
20047,1.0,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...,teambarcelona ..you look lost so you should fo...,miss people still never want see
20048,0.0,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...,anti-statist; i homeschool my kids. aspiring t...,"bitemyapp noticed tendency pee carpet, want sa..."


In [186]:
df.isnull().sum()

gender             5030
description           0
text                  0
descrip_Cleaned       0
text_Cleaned          0
dtype: int64

In [187]:
df.dropna(axis = 0, inplace = True)

In [188]:
df.isnull().sum()

gender             0
description        0
text               0
descrip_Cleaned    0
text_Cleaned       0
dtype: int64

In [189]:
df['descrip_Cleaned'] = df['descrip_Cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


# Dropping of redundant columns ‘description’ and ‘text’ columns were dropped

In [190]:
df.drop(['description','text'],axis=1,inplace=True)
df.head()

,gender,descrip_Cleaned,text_Cleaned
0,1.0,sing rhythm.,robbie e responds critics win eddie edwards wo...
1,1.0,i'm author novels filled family drama romance.,felt like friends living story
2,1.0,louis whining squealing,absolutely adore louis starts songs hits hard ...
3,1.0,"mobile guy. 49ers, shazam, google, kleiner per...",hi jordanspieth - looking url - use ifttt?! ty...
4,0.0,ricky wilson best frontman/kaiser chiefs best ...,watching neighbours sky catching neighbs!! xxx...


In [191]:
df.isnull().sum()

gender             0
descrip_Cleaned    0
text_Cleaned       0
dtype: int64

# Vectorization of texts in ‘descrip_Cleaned’ and ‘text_Cleaned’ columns.

In [192]:
x= df['text_Cleaned']


In [193]:
y = df['gender']


In [201]:
cv = CountVectorizer(max_features = 1000)
x = cv.fit_transform(df['descrip_Cleaned']).toarray()
x1=cv.fit_transform(df['text_Cleaned']).toarray()


In [202]:
A=pd.DataFrame(x)

B=pd.DataFrame(x1)

In [209]:
A.shape

(11194, 1000)

In [210]:
B.shape

(11194, 1000)

In [203]:
X=pd.concat([B,A],join='outer',axis=1)
X.shape

(11194, 2000)

In [204]:
x = np.array(X)

# Data was split into train and test data with an 80:20 ratio.

In [205]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

In [206]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [196]:
# Vectorize text reviews to numbers
#vec = CountVectorizer(stop_words='english')
#x_train = vec.fit_transform(X_train).toarray()
#x_test = vec.transform(X_test).toarray()

In [224]:
gnbmodel = GaussianNB()
gnbmodel.fit(X_train , y_train)
y_pred = gnbmodel.predict(X_test)

In [208]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 64.09%


In [211]:
confusion_matrix(y_test, y_pred)


array([[757, 375],
       [429, 678]], dtype=int64)

In [212]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.64      0.67      0.65      1132
         1.0       0.64      0.61      0.63      1107

    accuracy                           0.64      2239
   macro avg       0.64      0.64      0.64      2239
weighted avg       0.64      0.64      0.64      2239



# GridsearchCV() was used to tune hyperparameters for three classifiers.

In [213]:
param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

In [214]:
nbModel_grid = GridSearchCV(estimator=gnbmodel, param_grid=param_grid_nb, verbose=1, cv=3, n_jobs=-1)


In [215]:
nbModel_grid.fit(X_train, y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


GridSearchCV(cv=3, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.8480358...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=1)

In [216]:
nbModel_grid.best_params_

{'var_smoothing': 0.01873817422860384}

In [217]:
y_pred_hyper = nbModel_grid.predict(X_test)

In [218]:
print(confusion_matrix(y_test, y_pred_hyper), ": is the confusion matrix")

[[879 253]
 [512 595]] : is the confusion matrix


In [244]:
accuracy = accuracy_score(y_test, y_pred_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 65.83%


In [219]:
print(classification_report(y_test, y_pred_hyper))

              precision    recall  f1-score   support

         0.0       0.63      0.78      0.70      1132
         1.0       0.70      0.54      0.61      1107

    accuracy                           0.66      2239
   macro avg       0.67      0.66      0.65      2239
weighted avg       0.67      0.66      0.65      2239



In [249]:
# save the model to disk
filenameRF = 'model_pt_f.sav'


pickle.dump(nbModel_grid, open(filenameRF, 'wb'))

# Lightgbm Classifier

In [225]:
lgbmodel = LGBMClassifier(max_depth=3)
lgbmodel.fit(X_train, y_train)

LGBMClassifier(max_depth=3)

In [226]:
y_pred1= lgbmodel.predict(X_test)

In [230]:
accuracy = accuracy_score(y_test, y_pred1)

print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 61.32%


# Hyper parameter tuning using Grid Search CV

In [231]:
param_grid = {
              "max_depth": [2, 3, 5, 10],
              "min_child_weight": [0.001, 0.002],
              "learning_rate": [0.05, 0.1]
              }

In [232]:
lgbgrid = GridSearchCV(estimator = lgbmodel, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 0)

lgbgrid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LGBMClassifier(max_depth=3), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1],
                         'max_depth': [2, 3, 5, 10],
                         'min_child_weight': [0.001, 0.002]})

In [233]:
lgbgrid.best_params_

{'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 0.001}

In [234]:
y_pred1_hyper = lgbgrid.predict(X_test)

In [235]:
accuracy = accuracy_score(y_test, y_pred1_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 63.82%


# XGBoost Classifier

In [236]:
xgbmodel = XGBClassifier(max_depth=5, min_child_weight=1)
xgbmodel.fit(X_train, y_train)

[12:08:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [248]:
# save the model to disk
filenameRF = 'model_pt.sav'


pickle.dump(xgbmodel, open(filenameRF, 'wb'))

In [237]:
y_pred2 = xgbmodel.predict(X_test)


In [238]:
accuracy = accuracy_score(y_test, y_pred2)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 64.31%


# Hyperparameter tuning of XGBoost Classifier using GridsearchCV() function.

In [239]:
xgb_param_grid = {
              "max_depth": [3, 5],
              "min_child_weight": [1, 2],
              }

In [240]:
xgbgrid = GridSearchCV(estimator = xgbmodel, param_grid = xgb_param_grid, cv = 3, n_jobs = -1, verbose = 0)

xgbgrid.fit(X_train, y_train)

[12:17:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=5,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_estimators=100, n_jobs=4,
                                     num_parallel_tree=1, random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_method='exact', validate_parameters=1,
         

In [241]:
xgbgrid.best_params_

{'max_depth': 5, 'min_child_weight': 2}

In [242]:
y_pred2_hyper = xgbgrid.predict(X_test)


In [250]:
y_pred2_hyper

array([0., 0., 1., ..., 0., 0., 0.])

In [243]:
accuracy = accuracy_score(y_test, y_pred2_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 64.63%
